# OpenAI Function Calling Demo

This notebook demonstrates how to use OpenAI's function calling capability to create and use tools.

In [1]:
pip install oauth2client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gspread

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install functions

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install llm_connect

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import sys

# adding the directory with the connection code to the system path
# so that we can import it
sys.path.append("../code")

In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

/Users/nujoumunus/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
# this cell isn't strictly necessary but it makes it easier to
# to update the code from llm_connect if you make changes
# %reload_ext autoreload
# %autoreload 2
# %aimport functions

In [9]:
import json
import requests
from langchain_openai import ChatOpenAI
# from llm_connect import get_response
from dotenv import load_dotenv
import functions as fn

load_dotenv()


SyntaxError: invalid syntax (functions.py, line 68)

In [10]:
import json
import requests
from langchain_openai import ChatOpenAI
# from llm_connect import get_response
from dotenv import load_dotenv
# import functions as fn

load_dotenv()

True

In [11]:
client = ChatOpenAI(
    model="llama-3",
    api_key=os.getenv("LLAMA_API_KEY"),
    openai_api_base="https://traip13.tgptinf.ucsd.edu/v1",
)

#### Vendor Suggestion Tool

In [ ]:
def get_data(sheet_name: str) -> list:
    """
    Fetch data  from a Google Sheet.

    Parameters
    ----------
    sheet_name : str
        Name of the Google Sheet tab.

    Returns
    -------
    list of dict
        Each row in the sheet as a dictionary.
    """
    scope = [
    "https://spreadsheets.google.com/feeds", 
    "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
    gs_client = gspread.authorize(creds)  
    spreadsheet = gs_client.open_by_url("https://docs.google.com/spreadsheets/d/1ioqw78lMaf1tpFrU4QmcShZgl79bn6iqljnF3Uu8krU/edit?gid=244310017#gid=244310017/edit#gid=0")
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    return data

In [16]:
def get_item_list_from_sheet(sheet_name: str = "items") -> list:
    raw_data = get_data(sheet_name)
    return [row["Item Name"] for row in raw_data if "Item Name" in row and row["Item Name"].strip()]
items = get_item_list_from_sheet("items")

ValueError: ('Unexpected credentials type', None, 'Expected', 'service_account')

In [ ]:
def get_item_data_from_sheet(sheet_name: str = "items") -> list:
    """
    Fetches full item rows from the Google Sheet tab 'items'.

    Returns
    -------
    list of dicts:
        Each dict contains item metadata like:
        Item Name, Specification, Quantity, Unit of Measure, Delivery Date, Drawing Ref
    """
    return get_data(sheet_name)


In [ ]:
item_data = get_item_data_from_sheet("items")


In [ ]:
def vendor_selector_template(user_input):
    return f"""
You are a procurement assistant tasked with selecting the most suitable vendor for each item listed.

You have access to vendor profiles that include:

Past history with our company

Relevant certifications

Area of expertise

Average lead time (in days)

Your task:
For each item, select the single best vendor based on the above criteria and the relevance of the vendor's profile to the item’s requirements.

Output Format:
Return the results in a markdown table with the following columns:

Item

Suggested Vendor

Reason for Selection (1–2 bullet points per row)

Additional Instructions:

If the same vendor is suitable for multiple items, list those item names together in the same row under the “Item” column (comma-separated).

Do not include any introductory or closing text.

Add a title above the table:
"Vendors suggestion based on Bechtel database"" 

Item to source: {user_input['item']}

Vendor list:
{user_input['vendor_data']}
"""

def suggest_vendors_llm(item_name: list, llm: str = "llama") -> str:
    vendor_data = get_data("vendors")

    prompt_input = {
        "item": item_name,
        "vendor_data": vendor_data
    }

    return get_response(
        input=prompt_input,
        template=vendor_selector_template,
        role="You are a procurement assistant helping select vendors for a specific item.",
        temperature=0.4,
        max_tokens=400,
        md=True,
        llm=llm
    )


In [ ]:
suggest_vendors_llm(items)

# Vendors suggestion based on Bechtel database
| Item | Suggested Vendor | Reason for Selection |
| --- | --- | --- |
| Carbon Steel Flange, Gate Valve | Alpha Solutions 2 | * Expertise in Flanges and Valves, * Certifications include API Monogram and CE |
| HVAC Duct Connector | Trans Partners 13 | * Expertise in HVAC Duct Connectors, * Fast lead time of 12 days |
| Cable Tray, Signal Conduit, Control Cable | Metro Industries 1 | * Expertise in Control Cable, Signal Conduit, and Cable Tray, * Preferred vendor for 3 years |
| Earthing Strip, Fire Rated Door | Alpha Solutions 2 | * Expertise in Earthing Strip and Fire Rated Door, * Certifications include API Monogram and CE |
| Structural Steel Beam | Steel Solutions 16 | * Expertise in Structural Steel, * Preferred vendor for 3 years |
| Lighting Fixture - Tunnel Type | Metro Industries 9 | * Expertise in Lighting Fixture, * Fast lead time of 13 days |

#### RFQ Formatter Tool

In [ ]:
def generate_rfq_document(project_info: dict, item_data: list, vendor_name: str = None) -> str:
    """
    Generate a vendor-ready RFQ document in plain text/markdown.

    Parameters
    ----------
    project_info : dict
        Contains project name, delivery address, submission deadline, and buyer contact info.

    item_data : list of dict
        List of items to be included in the RFQ. Each item should have fields:
        - Item Name, Specification, Quantity, Unit of Measure, Delivery Date, Drawing Ref

    vendor_name : str, optional
        If a specific vendor is being addressed, include their name in the greeting.

    Returns
    -------
    str
        A formatted RFQ text block ready to be emailed or saved.
    """


In [ ]:
def rfq_formatter_prompt(inputs):
    vendor_name = inputs.get("vendor_name", "Vendor")
    project = inputs["project_info"]
    items = inputs["item_data"]

    item_lines = "\n".join(
        [f"- {item['Item Name']}: {item['Specification']} | Qty: {item['Quantity']} {item['Unit of Measure']} | Delivery by: {item['Delivery Date']} | Ref: {item['Drawing Ref']}"
         for item in items]
    )

    return f"""
You are a procurement assistant tasked with creating a vendor-ready RFQ.
Generate a professional, well-structured RFQ in markdown format with the following structure:

Greeting:
"Dear {vendor_name},"

Body:
- Brief project description using: {project['Project Name']}, located in {project['Location']}
- Include buyer contact info: {project['Engineering Lead']} (procurement@doha-metro.com)
- List the items requested as bullet points

Closing:
- State deadline for quotation submission: 1 week from today
- Ask the vendor to confirm receipt and reach out with questions

Item List:
{item_lines}

Only return the RFQ body. No commentary or explanation.
"""


In [ ]:
import json

# Load the JSON file
with open("/home/jovyan/git/rsm-mgta495-rsm-skaniappan-2/Sudhersan/project_context.json", "r") as file:
    project_info = json.load(file)

project_info

{'Project Name': 'Doha Metro Phase 2',
 'Project Code': 'QATAR-METRO-2025',
 'Location': 'Doha, Qatar',
 'Client': 'Qatar Rail',
 'Engineering Lead': 'Ahmed Al-Mansoori',
 'Budget Holder': 'Central Procurement Division',
 'Expected Completion': '2025-12-31',
 'Scope Area': ['Underground stations',
  'HVAC systems',
  'Piping systems',
  'Structural steel',
  'Cable trays',
  'Control rooms',
  'Lighting and electrical infrastructure'],
 'Working Language': 'English/Arabic',
 'Incoterms': 'CIF – Doha Port',
 'Payment Terms': 'Net 45',
 'Certifications Needed': ['ISO 9001',
  'ISO 14001',
  'QCS 2014',
  'GCC Standards'],
 'NDA Status': 'Signed'}

In [ ]:
suggested_vendor_map = {
    "Alpha Solutions 2": ["Carbon Steel Flange", "Gate Valve", "Earthing Strip", "Fire Rated Door"],
    "Trans Partners 13": ["HVAC Duct Connector"],
    "Metro Industries 1": ["Cable Tray", "Signal Conduit", "Control Cable"],
    "Steel Solutions 16": ["Structural Steel Beam"],
    "Metro Industries 9": ["Lighting Fixture - Tunnel Type"]
}

item_data_all = get_item_data_from_sheet("items")

# Generate RFQ per vendor
for vendor, item_names in suggested_vendor_map.items():
    item_subset = [item for item in item_data_all if item["Item Name"] in item_names]

    rfq_text = get_response(
        input={
            "project_info": project_info,
            "item_data": item_subset,
            "vendor_name": vendor
        },
        template=rfq_formatter_prompt,
        role="You are a helpful assistant drafting procurement RFQs.",
        llm="llama",
        md=True
    )

    print(f"\n--- RFQ for {vendor} ---\n")
    print(rfq_text)

### Request for Quotation for Doha Metro Phase 2 Project
Dear Alpha Solutions 2,

We are reaching out from the procurement department of Doha Metro Phase 2, located in Doha, Qatar, regarding a request for quotation for various mechanical and electrical items. For any inquiries or clarifications, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested:
* Carbon Steel Flange: ASTM A105, PN16, Raised Face | Qty: 150 EA | Delivery by: 2025-08-30 | Ref: DM-STN-MECH-001
* Gate Valve: API 600, 6-inch, WCB Body | Qty: 60 EA | Delivery by: 2025-08-30 | Ref: DM-STN-MECH-003
* Earthing Strip: Copper, 25mm x 3mm | Qty: 800 MTR | Delivery by: 2025-08-30 | Ref: DM-ELEC-018
* Fire Rated Door: 120 min fire resistance, single leaf, 900x2100mm | Qty: 35 EA | Delivery by: 2025-08-30 | Ref: DM-ARCH-021

Please submit your quotation within 1 week from the date of this request. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Alpha Solutions 2 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Trans Partners 13,

We are reaching out regarding the Doha Metro Phase 2 project, located in Doha, Qatar. For any inquiries or questions, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested for the project:
* HVAC Duct Connector: Flexible, 300mm x 300mm, Class 1 | Qty: 200 EA | Delivery by: 2025-08-30 | Ref: DM-HVAC-004

Please submit your quotation within 1 week from the date of this request. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Trans Partners 13 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Metro Industries 1,

We are reaching out from the procurement department of Doha Metro, regarding the Doha Metro Phase 2 project, located in Doha, Qatar. For any inquiries or clarifications, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested for the project:
* Cable Tray: Ladder Type, GI, 300mm x 50mm | Qty: 500 MTR | Delivery by: 2025-08-30 | Ref: DM-ELEC-015
* Control Cable: CU/PVC/PVC, 4C x 2.5 sq.mm | Qty: 1000 MTR | Delivery by: 2025-08-30 | Ref: DM-ELEC-022
* Signal Conduit: PVC, 50mm dia, ISI marked | Qty: 750 MTR | Delivery by: 2025-08-30 | Ref: DM-SIG-005

Please submit your quotation within 1 week from the date of this RFQ. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Metro Industries 1 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Steel Solutions 16,

We are reaching out from the procurement department of Doha Metro Phase 2, located in Doha, Qatar, regarding a potential supply opportunity. For any inquiries or clarifications, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested for quotation:
* Structural Steel Beam: IPE 300, Hot Rolled, EN 10025 | Qty: 120 MTR | Delivery by: 2025-08-30 | Ref: DM-STRC-002

Please submit your quotation within 1 week from the date of this request. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Steel Solutions 16 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Metro Industries 9,

We are reaching out to request a quotation for the supply of materials for the Doha Metro Phase 2 project, located in Doha, Qatar. For any inquiries or clarification regarding this RFQ, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested:
* Lighting Fixture - Tunnel Type: IP66, LED, 230V, 4000K | Qty: 220 EA | Delivery by: 2025-08-30 | Ref: DM-LGT-010

Please submit your quotation within 1 week from the date of this letter. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Metro Industries 9 ---

None
